# **Method 2: Personalized Client Selection(NON IID)**

This method implements personalized client selection by dynamically adjusting selection probabilities to prioritize underperforming clients based on their probabilty, using the formula 1 - accuracy.

In [ ]:
# Install necessary packages
!pip install flwr
!pip install torch
!pip install torchvision
!pip install matplotlib

In [ ]:
# Import necessary modules
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision import datasets

import flwr as fl
from flwr.client import NumPyClient
from flwr.common import Metrics, FitRes, EvaluateRes, Parameters, Scalar, Config
from flwr.server.strategy import FedAvg
from flwr.simulation import start_simulation

In [ ]:
# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")
print(f"Flower {fl.__version__} / PyTorch {torch.__version__}")

import sys
import logging

logging.disable(sys.maxsize)

Training on cpu
Flower 1.14.0 / PyTorch 2.5.1+cu121


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Global variables
NUM_CLIENTS = 10
BATCH_SIZE = 32
NUM_ROUNDS = 20
LOCAL_EPOCHS = 1

# Define the neural network model for MNIST
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)   # 1 input channel for grayscale imgs
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 16 * 4 * 4 = 256
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
# Define the training function
def train(net, trainloader, epochs: int, verbose=False):

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch[0].to(DEVICE), batch[1].to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item() * labels.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")

# Define the testing function
def test(net, testloader):

    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch[0].to(DEVICE), batch[1].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
# Define parameter management functions
def set_parameters(net, parameters: List[np.ndarray]):

    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

def get_parameters(net) -> List[np.ndarray]:

    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
# Define the Flower client
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config: Dict) -> List[np.ndarray]:
        return get_parameters(self.net)

    def fit(self, parameters: List[np.ndarray], config: Dict) -> Tuple[List[np.ndarray], int, Dict]:
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=LOCAL_EPOCHS)
        return get_parameters(self.net), len(self.trainloader.dataset), {}

    def evaluate(self, parameters: List[np.ndarray], config: Dict) -> Tuple[float, int, Dict]:
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

In [ ]:
# Define the client function
def client_fn(cid: str) -> fl.client.Client:

    # Load model
    net = Net().to(DEVICE)

    # Use client ID as partition ID (ensure it's an integer)
    partition_id = int(cid)
    trainloader, valloader = client_train_loaders[partition_id], client_val_loaders[partition_id]


    return FlowerClient(net, trainloader, valloader)

In [ ]:
# Data Partitioning Function for MNIST
def create_non_iid_partitions(train_dataset, num_clients=10):

    from torch.utils.data import Subset
    import numpy as np

    # Initialize a dict to hold indices for each client
    client_indices = {i: [] for i in range(num_clients)}

    # Define the clients and their class assignments
    # Clients 0-1: classes 0,1,2
    # Clients 2-3: classes 3,4,5
    # Clients 4-5: classes 6,7,8,9
    # Clients 6-9: all classes with different distributions

    # Define class groups
    class_groups = {
        0: [0,1,2],
        1: [0,1,2],
        2: [3,4,5],
        3: [3,4,5],
        4: [6,7,8,9],
        5: [6,7,8,9],
        6: list(range(10)),
        7: list(range(10)),
        8: list(range(10)),
        9: list(range(10)),
    }

    # Get all indices per class
    class_indices = {i: [] for i in range(10)}
    for idx, (img, label) in enumerate(train_dataset):
        class_indices[label].append(idx)

    # Shuffle the indices within each class
    for cls in class_indices:
        np.random.shuffle(class_indices[cls])

    # Assign data to clients 0-5 based on their class groups
    for client_id in range(6):
        assigned_classes = class_groups[client_id]
        for cls in assigned_classes:
            # Assign a proportion of the class data to this client
            # For simplicity, split equally among clients in the same group
            num_clients_in_group = 2
            num_samples = int(len(class_indices[cls]) / num_clients_in_group)
            client_indices[client_id].extend(class_indices[cls][:num_samples])
            class_indices[cls] = class_indices[cls][num_samples:]

    # Assign data to clients 6-9 with all classes but different distributions
    # Define different distributions for clients 6-9
    # Each client has a different emphasis on certain classes



    client_distribution_weights = {
        6: [0.4, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
        7: [0.1, 0.4, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
        8: [0.05, 0.05, 0.4, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
        9: [0.05, 0.05, 0.05, 0.4, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
    }

    for client_id in range(6, 10):
        weights = client_distribution_weights[client_id]
        for cls in range(10):
            if len(class_indices[cls]) == 0:
                continue
            # Determine number of samples to assign based on weight
            num_samples = int(weights[cls] * len(class_indices[cls]))
            # Ensure at least one sample is assigned if available
            if num_samples == 0 and len(class_indices[cls]) > 0:
                num_samples = 1
            # Assign samples to client
            client_indices[client_id].extend(class_indices[cls][:num_samples])
            class_indices[cls] = class_indices[cls][num_samples:]

    # Assign any remaining samples to clients 6-9
    for cls in range(10):
        remaining_indices = class_indices[cls]
        for idx in remaining_indices:
            client_id = idx % 4 + 6  # Clients 6,7,8,9
            client_indices[client_id].append(idx)

    # Create Subsets for each client
    client_subsets = []
    for client_id in range(num_clients):
        subset = Subset(train_dataset, client_indices[client_id])
        client_subsets.append(subset)

    # For each client, create a validation loader
    client_train_loaders = []
    client_val_loaders = []
    for subset in client_subsets:
        num_train = int(0.8 * len(subset))
        num_val = len(subset) - num_train
        train_subset, val_subset = torch.utils.data.random_split(
            subset, [num_train, num_val],
            generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(
            train_subset, batch_size=BATCH_SIZE, shuffle=True
        )
        val_loader = DataLoader(
            val_subset, batch_size=BATCH_SIZE, shuffle=False
        )
        client_train_loaders.append(train_loader)
        client_val_loaders.append(val_loader)

    return client_train_loaders, client_val_loaders

# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

# Download and load the training dataset
train_dataset = datasets.MNIST(root='./data', train=True,
                                 download=True, transform=transform)

# Download and load the test dataset
test_dataset = datasets.MNIST(root='./data', train=False,
                                download=True, transform=transform)

# Create a global test loader with all classes equally distributed
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Create non-iid partitions for clients
client_train_loaders, client_val_loaders = create_non_iid_partitions(train_dataset, num_clients=NUM_CLIENTS)

Personalised strategy

In [ ]:
# Define the custom federated averaging strategy with personalized client selection
class CustomFed(FedAvg):
    def __init__(self, max_clients_per_round: int, **kwargs):
        super().__init__(**kwargs)
        self.max_clients_per_round = max_clients_per_round
        self.client_accuracies: Dict[str, float] = {}
        self.metrics_distributed = {"accuracy": []}
        self.latest_parameters: Optional[Parameters] = None

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: fl.server.client_manager.ClientManager,
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitIns]]:
        # Get all available clients
        available_clients = list(client_manager.all().values())

        if server_round == 1:
            # First round: select clients at random
            sample_size = min(self.max_clients_per_round, len(available_clients))
            selected_clients = np.random.choice(available_clients, size=sample_size, replace=False)
            selected_clients = list(selected_clients)
            print(f"Round {server_round}: Selected clients {[client.cid for client in selected_clients]}")
        else:
            # Compute selection probabilities based on previous accuracies
            client_ids = [client.cid for client in available_clients]
            accuracies = np.array([self.client_accuracies.get(cid, 0.0) for cid in client_ids])
            probabilities = 1.0 - accuracies  # p[k, t+1] = 1 - a[k, t]
            total_prob = probabilities.sum()
            if total_prob == 0:
                # If all accuracies are 1.0, assign equal probabilities
                probabilities = np.ones_like(probabilities) / len(probabilities)
            else:
                probabilities /= total_prob  # Normalize to sum to 1
            # Select clients based on probabilities
            sample_size = min(self.max_clients_per_round, len(available_clients))
            selected_indices = np.random.choice(
                len(available_clients),
                size=sample_size,
                replace=False,
                p=probabilities,
            )
            selected_clients = [available_clients[i] for i in selected_indices]
            print(f"Round {server_round}: Selection probabilities {dict(zip(client_ids, probabilities.round(3)))}")
            print(f"Round {server_round}: Selected clients {[client.cid for client in selected_clients]}")

        # Create fit instructions
        config = {}
        fit_ins = fl.common.FitIns(parameters, config)
        # Return list of clients and fit instructions
        return [(client, fit_ins) for client in selected_clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Call the superclass method to perform aggregation
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            # Store the latest global parameters
            self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Optional[float]:
        # Collect accuracies and number of examples from clients
        metrics = []
        client_accuracies_round = {}
        for client_proxy, evaluate_res in results:
            cid = client_proxy.cid
            num_examples = evaluate_res.num_examples
            accuracy = evaluate_res.metrics.get("accuracy", 0.0)
            self.client_accuracies[cid] = accuracy
            client_accuracies_round[cid] = accuracy
            metrics.append((num_examples, {"accuracy": accuracy}))
        # Compute weighted average accuracy
        aggregated_metrics = self.weighted_average(metrics)
        global_accuracy = aggregated_metrics["accuracy"]
        # Store global accuracy for plotting
        self.metrics_distributed["accuracy"].append((server_round, global_accuracy))
        # Log the global accuracy and individual client accuracies
        print(f"Round {server_round} aggregated accuracy: {global_accuracy:.4f}")
        print(f"Round {server_round} client accuracies: {client_accuracies_round}")
        # Proceed with default aggregation
        return super().aggregate_evaluate(server_round, results, failures)

    @staticmethod
    def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
        # Calculate weighted average of accuracy
        accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
        examples = [num_examples for num_examples, _ in metrics]
        return {"accuracy": sum(accuracies) / sum(examples) if sum(examples) > 0 else 0.0}

In [ ]:
!pip install -U "flwr[simulation]"

In [ ]:
# Initialize the custom FedAvg strategy
custom_strat = CustomFed(
    max_clients_per_round=5,  # Set the maximum number of clients per round
    fraction_fit=None,
    min_fit_clients=5,        # Ensure at least 5 clients are selected
    min_available_clients=NUM_CLIENTS,
)

# Start the simulation with the custom strategy
print("\nStarting simulation with CustomFedAvg (Personalized Client Selection)...")
history_custom = start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    client_resources={"num_gpus": 1 if torch.cuda.is_available() else 0},
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=custom_strat,
)


Starting simulation with CustomFedAvg (Personalized Client Selection)...


<ipython-input-13-8fcfb6d56851>:11: UserWarning: No `num_cpus` specified in `client_resources`. Using `num_cpus=1` for each client.
  history_custom = start_simulation(
(pid=6081) 2024-12-27 21:52:44.098414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=6081) 2024-12-27 21:52:44.139316: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=6081) 2024-12-27 21:52:44.156996: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=6081) 2024-12-27 21:52:46.426918: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Round 1: Selected clients ['4050256589846675593', '17351513886539913398', '8028081821430494994', '17974523050370469186', '13082124077522875752']


(ClientAppActor pid=6082) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=6082) 
(ClientAppActor pid=6082)             This is a deprecated feature. It will be removed
(ClientAppActor pid=6082)             entirely in future versions of Flower.
(ClientAppActor pid=6082)         
(ClientAppActor pid=6082) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(pid=6082) 2024-12-27 21:52:44.098396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=6082) 2024-12-27 21:52:44.139544: 

Round 1 aggregated accuracy: 0.2850
Round 1 client accuracies: {'8028081821430494994': 0.2884882108183079, '17351513886539913398': 0.33661971830985915, '11970180661151594236': 0.960919540229885, '13776929052930500947': 0.0, '3063427337728765025': 0.0, '13082124077522875752': 0.9626436781609196, '4050256589846675593': 0.25882352941176473, '14886421105598068772': 0.020397208803005905, '126186853608214035': 0.2642679900744417, '17974523050370469186': 0.015037593984962405}
Round 2: Selection probabilities {'14886421105598068772': 0.142, '17974523050370469186': 0.143, '13082124077522875752': 0.005, '11970180661151594236': 0.006, '13776929052930500947': 0.145, '3063427337728765025': 0.145, '126186853608214035': 0.107, '4050256589846675593': 0.108, '8028081821430494994': 0.103, '17351513886539913398': 0.096}
Round 2: Selected clients ['17974523050370469186', '126186853608214035', '8028081821430494994', '3063427337728765025', '4050256589846675593']


...

Round 18 aggregated accuracy: 0.8461
Round 18 client accuracies: {'126186853608214035': 0.8374689826302729, '13082124077522875752': 0.7350574712643678, '17351513886539913398': 0.8718309859154929, '14886421105598068772': 0.7691894793344068, '13776929052930500947': 0.9924937447873228, '4050256589846675593': 0.857516339869281, '3063427337728765025': 0.9933277731442869, '8028081821430494994': 0.8072122052704577, '11970180661151594236': 0.7206896551724138, '17974523050370469186': 0.7669172932330827}
Round 19: Selection probabilities {'14886421105598068772': 0.14, '17974523050370469186': 0.141, '13082124077522875752': 0.161, '11970180661151594236': 0.169, '13776929052930500947': 0.005, '3063427337728765025': 0.004, '126186853608214035': 0.099, '4050256589846675593': 0.086, '8028081821430494994': 0.117, '17351513886539913398': 0.078}
Round 19: Selected clients ['11970180661151594236', '13082124077522875752', '4050256589846675593', '8028081821430494994', '126186853608214035']


(ClientAppActor pid=6082)          [repeated 12x across cluster]
(ClientAppActor pid=6082) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 6x across cluster]
(ClientAppActor pid=6082)             This is a deprecated feature. It will be removed [repeated 6x across cluster]
(ClientAppActor pid=6082)             entirely in future versions of Flower. [repeated 6x across cluster]
(ClientAppActor pid=6082) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 6x across cluster]
(ClientAppActor pid=6081)          [repeated 6x across cluster]
(ClientAppActor pid=6081) WARNING :   DEPRECATED FEATURE: `client_fn` now exp

Round 19 aggregated accuracy: 0.8853
Round 19 client accuracies: {'11970180661151594236': 0.993103448275862, '13776929052930500947': 0.8815679733110926, '17351513886539913398': 0.928169014084507, '13082124077522875752': 0.996551724137931, '3063427337728765025': 0.8824020016680567, '4050256589846675593': 0.9124183006535947, '14886421105598068772': 0.7869028448738594, '17974523050370469186': 0.7926960257787325, '8028081821430494994': 0.8321775312066574, '126186853608214035': 0.8622828784119106}
Round 20: Selection probabilities {'14886421105598068772': 0.188, '17974523050370469186': 0.183, '13082124077522875752': 0.003, '11970180661151594236': 0.006, '13776929052930500947': 0.105, '3063427337728765025': 0.104, '126186853608214035': 0.122, '4050256589846675593': 0.077, '8028081821430494994': 0.148, '17351513886539913398': 0.063}
Round 20: Selected clients ['14886421105598068772', '3063427337728765025', '17974523050370469186', '4050256589846675593', '8028081821430494994']


(ClientAppActor pid=6081)          [repeated 12x across cluster]
(ClientAppActor pid=6081) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 6x across cluster]
(ClientAppActor pid=6081)             This is a deprecated feature. It will be removed [repeated 6x across cluster]
(ClientAppActor pid=6081)             entirely in future versions of Flower. [repeated 6x across cluster]
(ClientAppActor pid=6081) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 6x across cluster]
(ClientAppActor pid=6081)          [repeated 6x across cluster]
(ClientAppActor pid=6081) WARNING :   DEPRECATED FEATURE: `client_fn` now exp

Round 20 aggregated accuracy: 0.9647
Round 20 client accuracies: {'4050256589846675593': 0.9738562091503268, '126186853608214035': 0.9441687344913151, '11970180661151594236': 0.9356321839080459, '8028081821430494994': 0.9611650485436893, '17974523050370469186': 0.9957035445757251, '13776929052930500947': 0.9570475396163469, '17351513886539913398': 0.9774647887323944, '3063427337728765025': 0.9549624687239366, '13082124077522875752': 0.9511494252873564, '14886421105598068772': 0.9930220075147611}


Evaluating for test set

In [ ]:
# Evaluate the final global model from the custom strategy
final_parameters_custom = custom_strat.latest_parameters

def evaluate_global_model(global_parameters: Parameters, testloader):


    params = fl.common.parameters_to_ndarrays(global_parameters)

    # Initialize a new model instance
    net = Net().to(DEVICE)

    # Set the model parameters to the global parameters
    set_parameters(net, params)

    # Evaluate on the test set
    loss, accuracy = test(net, testloader)
    return loss, accuracy

if final_parameters_custom is not None:
    loss_custom, accuracy_custom = evaluate_global_model(final_parameters_custom, testloader)
    print(f"\nFinal test set performance after personalized selection:")
    print(f"\tLoss: {loss_custom:.4f}")
    print(f"\tAccuracy: {accuracy_custom:.4f}")
else:
    print("Final global parameters from CustomFedAvg are not available.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Final test set performance after personalized selection:
	Loss: 0.1091
	Accuracy: 0.9661


Uniform strategy to compare with personalised one

In [ ]:
# Define a separate FlowerClient class for the standard strategy to avoid conflicts
class FlowerClientStandard(NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config: Dict) -> List[np.ndarray]:
        return get_parameters(self.net)

    def fit(self, parameters: List[np.ndarray], config: Dict) -> Tuple[List[np.ndarray], int, Dict]:
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=LOCAL_EPOCHS)
        return get_parameters(self.net), len(self.trainloader.dataset), {}

    def evaluate(self, parameters: List[np.ndarray], config: Dict) -> Tuple[float, int, Dict]:
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}


In [ ]:

def client_fn_standard(cid: str) -> fl.client.Client:

    # Load model
    net = Net().to(DEVICE)

    # Use client ID as partition ID
    partition_id = int(cid)
    trainloader, valloader = client_train_loaders[partition_id], client_val_loaders[partition_id]


    return FlowerClientStandard(net, trainloader, valloader)


In [ ]:
# Define the standard FedAvg strategy with parameter storage
class FedAvgWithParams(FedAvg):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.latest_parameters: Optional[Parameters] = None  # To store the latest global parameters
        self.metrics_distributed = {"accuracy": []}         # Store global accuracy over rounds

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Perform the default aggregation
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            # Store the latest global parameters
            self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Optional[float]:
        # Collect accuracies and number of examples from clients
        metrics = []
        for client_proxy, evaluate_res in results:
            num_examples = evaluate_res.num_examples
            accuracy = evaluate_res.metrics.get("accuracy", 0.0)
            metrics.append((num_examples, {"accuracy": accuracy}))
        # Compute weighted average accuracy
        aggregated_metrics = self.weighted_average(metrics)
        global_accuracy = aggregated_metrics["accuracy"]
        # Store global accuracy for plotting
        self.metrics_distributed["accuracy"].append((server_round, global_accuracy))
        # Log the global accuracy
        print(f"Round {server_round} aggregated accuracy: {global_accuracy:.4f}")

        return super().aggregate_evaluate(server_round, results, failures)

    @staticmethod
    def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
        # Calculate weighted average of accuracy
        accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
        examples = [num_examples for num_examples, _ in metrics]
        return {"accuracy": sum(accuracies) / sum(examples) if sum(examples) > 0 else 0.0}

In [ ]:
# Initialize the standard FedAvg strategy
standard_strategy = FedAvgWithParams(
    fraction_fit=0.5,         # m/N = 5/10 = 0.5
    min_fit_clients=5,        # Ensure at least 5 clients are selected
    min_available_clients=NUM_CLIENTS,
)

# Start the simulation with the standard FedAvg strategy
print("\nStarting simulation with FedAvg (Uniform Client Selection)...")
history_standard = start_simulation(
    client_fn=client_fn_standard,
    num_clients=NUM_CLIENTS,
    client_resources={"num_gpus": 1 if torch.cuda.is_available() else 0},
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=standard_strategy,
)


Starting simulation with FedAvg (Uniform Client Selection)...


<ipython-input-17-32bc23eb5ee4>:10: UserWarning: No `num_cpus` specified in `client_resources`. Using `num_cpus=1` for each client.
  history_standard = start_simulation(
(pid=8369) 2024-12-27 21:59:56.496332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=8369) 2024-12-27 21:59:56.547838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=8369) 2024-12-27 21:59:56.559611: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=8371) 2024-12-27 22:00:00.097161: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(ClientAppActor pid=8371) 

Round 1 aggregated accuracy: 0.3224


(ClientAppActor pid=8369) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=8369)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=8369)             entirely in future versions of Flower. [repeated 4x across cluster]
(ClientAppActor pid=8369) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
(ClientAppActor pid=8369)          [repeated 8x across cluster]
(ClientAppActor pid=8371) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `

Round 2 aggregated accuracy: 0.5901


...

Round 18 aggregated accuracy: 0.9408


(ClientAppActor pid=8371)          [repeated 8x across cluster]
(ClientAppActor pid=8371) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=8371)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=8371)             entirely in future versions of Flower. [repeated 4x across cluster]
(ClientAppActor pid=8371) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
(ClientAppActor pid=8369)          [repeated 4x across cluster]
(ClientAppActor pid=8369) WARNING :   DEPRECATED FEATURE: `client_fn` now expe

Round 19 aggregated accuracy: 0.8749


(ClientAppActor pid=8369)          [repeated 12x across cluster]
(ClientAppActor pid=8369) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 6x across cluster]
(ClientAppActor pid=8369)             This is a deprecated feature. It will be removed [repeated 6x across cluster]
(ClientAppActor pid=8369)             entirely in future versions of Flower. [repeated 6x across cluster]
(ClientAppActor pid=8369) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 6x across cluster]
(ClientAppActor pid=8371)          [repeated 12x across cluster]
(ClientAppActor pid=8371) WARNING :   DEPRECATED FEATURE: `client_fn` now ex

Round 20 aggregated accuracy: 0.9105


Evaluating the uniform approach with test set

In [ ]:
# Evaluate the final global model from the uniform strategy
final_parameters_standard = standard_strategy.latest_parameters

if final_parameters_standard is not None:
    loss_standard, accuracy_standard = evaluate_global_model(final_parameters_standard, testloader)
    print(f"\nFinal test set performance after uniform selection:")
    print(f"\tLoss: {loss_standard:.4f}")
    print(f"\tAccuracy: {accuracy_standard:.4f}")
else:
    print("Final global parameters from FedAvg are not available.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Final test set performance after uniform selection:
	Loss: 0.2711
	Accuracy: 0.9077
